## install & load packages

In [ ]:
!pip -q install git+https://github.com/mwshinn/PyDDM
import pyddm
import pyddm.plot
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pyddm import Sample

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## prep data

In [ ]:
# Load data - need to manually upload inference_tidy.csv into the files tab on each new runtime
df = pd.read_csv('inference_tidy.csv')
df = df.dropna(subset=['RT'])

# 8 drifts with memory thinning

In [ ]:
def drift_weights(t, trueCue, coherence, signal1_onset, memSampleRate, noise2_onset, signal2_onset,
                      mw80_noise1, mw50_noise1, mw80_signal1, mw50_signal1, mw80_noise2, mw50_noise2, mw80_signal2, mw50_signal2,
                      vw80_signal1, vw50_signal1, vw80_signal2, vw50_signal2):
    if t < signal1_onset:
        return trueCue/memSampleRate * (mw80_noise1 if abs(trueCue) == 0.8 else mw50_noise1)
    elif t < noise2_onset:
        return trueCue/memSampleRate * (mw80_signal1 if abs(trueCue) == 0.8 else mw50_signal1) + coherence * (vw80_signal1 if abs(trueCue) == 0.8 else vw50_signal1)
    elif t < signal2_onset:
        return trueCue/memSampleRate * (mw80_noise2 if abs(trueCue) == 0.8 else mw50_noise2)
    else:
        return trueCue/memSampleRate * (mw80_signal2 if abs(trueCue) == 0.8 else mw50_signal2) + coherence * (vw80_signal2 if abs(trueCue) == 0.8 else vw50_signal2)


# specify model
model = pyddm.gddm(
    drift = drift_weights,
    starting_position = 0,
    bound="B",
    T_dur = 4.1,
    nondecision='ndt',
    parameters={'B': (0.01, 3), 'ndt': (0, 0.5), 'memSampleRate': (1,150),
                'mw80_noise1': (-1, 1), 'mw50_noise1': (-1,1),
                'mw80_signal1': (-1, 1), 'mw50_signal1': (-1, 1),
                'mw80_noise2': (-1, 1), 'mw50_noise2': (-1,1),
                'mw80_signal2': (-1, 1), 'mw50_signal2': (-1, 1),
                 'vw80_signal1': (-1, 1), 'vw50_signal1': (-1, 1),
                'vw80_signal2': (-1, 1), 'vw50_signal2': (-1, 1)},
    conditions = ['trueCue', 'coherence', 'signal1_onset', 'noise2_onset', 'signal2_onset']
)


pyddm.plot.model_gui_jupyter(model, data_dt = 0.07, conditions={'trueCue': [-0.8, -0.65, 0.5, 0.65, 0.8],
                                                'coherence': [0.7],
                                                'trueCongruence': ['congruent', 'incongruent', 'neutral'],
                                                'signal1_onset': [0.3, 0.8],
                                                'noise2_onset': [1.2],
                                                'signal2_onset': [2.2]})

In [ ]:
len(df['signal1_onset'].unique())

np.linspace(0, 2.1, 25)

print(np.linspace(0.638, 1.26, 25), '\n\n', df['signal1_onset'].drop_duplicates().sort_values())

AttributeError: module 'numpy' has no attribute 'drop_duplicates'